# Sports Betting Arbitrage Identifier 
  
This notebook aims to identify arbitrage opportunities across a range of different book markers.  
This notebook analyses various bookmakers live market odds produce by `the-odds-api`  
  
Arbitrage opportunities are identified using the following techniques:  
1. N-event arbitrage (Head-to-head)  
2. Back-Lay event  
3. Lay all events  
4. Other markets  
  
For additional information on the above techniques, please explore the read.me or `utils.arb_search.get_arbs`  
  

### Notebook structure   
1. Read local inports  
2. Gather configs and establish global variables  
3. Test and gather active sports using the API  
4. Can either use   
    (1): Gather results for a single sport  
    (2): Gather results for all sports returned in step 3  
5. Explore payouts

In [ ]:
# Set up env
from utils import common, arb_search
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
# read .env file
config = common.read_yaml_config(
    file_path ="config.yaml",
)

API_KEY = config['config']['API_KEY']
API_URL = config['config']['API_URL']

In [ ]:
# Gather all sports
sports = common.get_sports_api(
    url=API_URL,
    api_key=API_KEY
)
# Sample response
sports[0]

## Explore arbs across a single sport

In [ ]:
# Example sport, can be changed to any available sport key in sports list
sport = 'soccer_epl'  
arbitrage_opportunities = []

# Get current market odds for the specified sport
current_market_odds = common.get_sports_api(
    url=f'{API_URL}/{sport}/odds',
    api_key=API_KEY,
    region='au',
    markets='h2h'
)
if 'error_code' in current_market_odds:
    logging.error(f"Error fetching odds: {current_market_odds['error_code']}. Message: {current_market_odds['message']}")
else:

    # Gather best markets and arbitrage opportunities
    market_details = arb_search.summary_market(current_market_odds)
    if market_details:
        for game in market_details:
            if game['arbitrage'] != []:
                arbitrage_opportunities.append(game)

# Print arbitrage opportunities
if arbitrage_opportunities:
    logging.info(f"Found {len(arbitrage_opportunities)} arbitrage opportunities:")
    arbitrage_opportunities
else:
    logging.info(f"No arbitrage opportunities found for {sport}.")

## Explore arbs across all available sports

In [ ]:
# Loop through each sport and gather current market odds
all_sports = []
best_markets = []
arbitrage_opportunities = []
x = 0
for sport in sports:
    sport_key = sport['key']

    try:
        current_market_odds = common.get_sports_api(
            url=f'{API_URL}/{sport_key}/odds',
            api_key=API_KEY,
            region='au',
            markets='h2h'
        )

        if 'error_code' not in current_market_odds:
            
            # Gather best markets and arbitrage opportunities
            all_sports.append(current_market_odds)
            market_details = arb_search.summary_market(current_market_odds)
            if market_details:
                best_markets.append(market_details)
                for game in market_details:
                    if game['arbitrage'] != []:
                        arbitrage_opportunities.append(game)

    except Exception as failed_request:
        logging.warning(f"Failed to get odds for {sport_key}: {failed_request}")
        pass


In [ ]:
# Print arbitrage opportunities
if arbitrage_opportunities:
    logging.info(f"Found {len(arbitrage_opportunities)} arbitrage opportunities:")
else:
    logging.info(f"No arbitrage opportunities found for any sport.")
arbitrage_opportunities

# Arbitrage Payouts

Below is an example of a Head-to-Head payout chart for a two-outcome event.

Fill in the **variable inputs** section below and use the slider to explore how different stake allocations affect your payout across both outcomes.

If you're interested in a full, in-depth payout chart for any type of bet please message my git hub account


In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown

# Input variables
team_a = "North Carolina Tar Heels"
team_b = "TCU Horned Frogs"
odds_A = 2.21
odds_B = 1.91
fee_A = 0
fee_B = 0
total_bet = 1000

constraint = round(total_bet / odds_A,0)
# --- Sliders for adjusting stakes ---
stake_A = widgets.FloatSlider(
    value=constraint,
    min=constraint - 50,
    max=total_bet,
    step=1,
    description='Stake A ($):',
    continuous_update=True
)

# # --- Output area ---
output = widgets.Output()

# --- Update function ---
def update_payouts(change=None):
    with output:
        output.clear_output()
        sA = stake_A.value
        sB = total_bet - sA
        payout_A = sA * odds_A
        payout_B = sB * odds_B

        profit_A = payout_A - total_bet
        profit_B = payout_B - total_bet
        roi_A = (profit_A / total_bet) * 100
        roi_B = (profit_B / total_bet) * 100

        minimum_stake_a = total_bet / odds_A
        minimum_stake_b = total_bet / odds_B
        maximum_stake_a = total_bet - minimum_stake_b
        maximum_stake_b = total_bet - minimum_stake_a

        display(Markdown(f"""
### Betting Constraints
- **Total Bet**: `${total_bet:.2f}`
- **{team_a}**: Min Stake - `${minimum_stake_a:.2f}` | Max Stake - `${maximum_stake_a:.2f}`
- **{team_b}**: Min Stake - `${minimum_stake_b:.2f}` | Max Stake - `${maximum_stake_b:.2f}`

---

### Profit Scenarios

| Bet         | Amount ($) | Payout Calculation          | Net Profit ($) | ROI (%) |
|-------------|------------|-----------------------------|----------------|---------|
| **{team_a}** Wins | {sA:.2f}    | {sA:.2f} × {odds_A} - {total_bet:.2f} = {payout_A:.2f} - {total_bet:.2f} | {profit_A:.2f}         | {roi_A:.2f}% |
| **{team_b}** Wins | {sB:.2f}    | {sB:.2f} × {odds_B} - {total_bet:.2f} = {payout_B:.2f} - {total_bet:.2f} | {profit_B:.2f}         | {roi_B:.2f}% |
        """))

stake_A.observe(update_payouts, names='value')

update_payouts()

display(stake_A, output) 